In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [2]:
with open('../1Classes/alchemist.txt',"r", encoding='utf-8') as f:
    text=f.read()

In [4]:
chars = sorted(set(text))
print(len(chars))
char2int = {ch: i for i, ch in enumerate(chars)}
int2char = {i: ch for i, ch in enumerate(chars)}

vocab = len(chars)

84


In [5]:
chars = sorted(set(text))
char2int = {ch: i for i, ch in enumerate(chars)}
int2char = {i: ch for i, ch in enumerate(chars)}

data_encoded = np.array([char2int[ch] for ch in text], dtype=np.int64)

class TextDataset(Dataset):
  def __init__(self, data, seq_length):
    self.data = data
    self.seq_length = seq_length

  def __len__(self):
    return len(self.data) - self.seq_length


  def __getitem__(self, idx):
    x = self.data[idx: idx+self.seq_length]
    y = self.data[idx+self.seq_length]
    x = torch.tensor(x, dtype=torch.long)
    y = torch.tensor(y, dtype=torch.long)
    return x, y


split = int(len(data_encoded)*0.8)

train_dataset = data_encoded[:split]
test_dataset = data_encoded[split:]

seq_len = 100
batch_size = 64
embed_size = 128
hidden_size = 256
num_layers = 2

train_dataset = TextDataset(train_dataset, seq_len)
test_dataset = TextDataset(test_dataset, seq_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)



class CharRNN(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.rnn = nn.RNN(input_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, vocab_size)
    self.hidden_size = hidden_size
    self.num_layers = num_layers

  def forward(self, x, hidden):
    emb = self.embedding(x) # [batch, seq] -> [batch, seq, embed_dim]
    out, hidden = self.rnn(emb, hidden) # out: [batch, seq, hidden]
    last = out[:, -1, :] # [batch, hidden]
    logits = self.fc(last)
    return logits, hidden

  def init_hidden(self, batch_size):
    return torch.zeros(self.num_layers, batch_size, self.hidden_size)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CharRNN(vocab_size=vocab, embed_size=embed_size, hidden_size=hidden_size, num_layers=num_layers).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epochs = 50

model.train()
for epoch in range(1, epochs+1):
  hidden = model.init_hidden(batch_size).to(device)
  total_loss = 0
  for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    optimizer.zero_grad()
    hidden = hidden.detach()
    logits, hidden = model(x_batch, hidden)
    loss = criterion(logits, y_batch)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

  print(f"Epoch: {epoch}, Loss: {total_loss/len(train_loader)}")

Epoch: 1, Loss: 1.7493509909615237
Epoch: 2, Loss: 1.442823504071539
Epoch: 3, Loss: 1.3604286251732738
Epoch: 4, Loss: 1.3164603889174953
Epoch: 5, Loss: 1.2880956370630818
Epoch: 6, Loss: 1.268713007774625
Epoch: 7, Loss: 1.2536734792976525
Epoch: 8, Loss: 1.2459016601203172
Epoch: 9, Loss: 1.237612524586168
Epoch: 10, Loss: 1.231329178474494
Epoch: 11, Loss: 1.2280090127631034
Epoch: 12, Loss: 1.2251963973418267
Epoch: 13, Loss: 1.2281517317653905
Epoch: 14, Loss: 1.2276080897544712
Epoch: 15, Loss: 1.2257580647544808
Epoch: 16, Loss: 1.2264776985637835
Epoch: 17, Loss: 1.2288291255011508
Epoch: 18, Loss: 1.2325118322483484
Epoch: 19, Loss: 1.23379922816436
Epoch: 20, Loss: 1.2371150639308257
Epoch: 21, Loss: 1.238876079763022
Epoch: 22, Loss: 1.2428622709034962
Epoch: 23, Loss: 1.2458061728992933
Epoch: 24, Loss: 1.2505912656248714
Epoch: 25, Loss: 1.2558982329605601
Epoch: 26, Loss: 1.2633340973840805
Epoch: 27, Loss: 1.261546203834877
Epoch: 28, Loss: 1.266388795713107
Epoch: 29,

In [8]:
# Text generate
def generated_text(model, start_text, length):
  model.eval()
  chars = [ch for ch in start_text]
  input_vector = torch.tensor([char2int[ch] for ch in chars], dtype=torch.long).unsqueeze(0).to(device)
  hidden = model.init_hidden(1).to(device)
  with torch.no_grad():
    for i in range(len(start_text)-1):
      logits, hidden = model(input_vector[:, i:i+1], hidden)
    last_char = input_vector[:, -1]


    for i in range(length):
      logits, hidden = model(last_char.unsqueeze(0), hidden) # logits-> (batch, seq, vocab_size)
      probs = torch.softmax(logits.squeeze(), dim=0)
      probs = probs.cpu().numpy()
      next_idx = np.random.choice(len(probs), p=probs)
      # next_idx = torch.argmax(probs).item()

      next_char = int2char[next_idx]
      chars.append(next_char)
      last_char = torch.tensor([next_idx], dtype=torch.long).to(device)

  return ''.join(chars)


text = 'But the caravan began to move, and it was impossible to hear what the Englishman was saying.'
generated = generated_text(model, text, 100)
print("Generated_text: ", generated)

Generated_text:  But the caravan began to move, and it was impossible to hear what the Englishman was saying.”
I’ve
baught, and, where the oasuible the old king
the starce how most
to explopreads, in the possi
